# Util Functions
---

- ### Formatar uma saída.

In [23]:
def get_formatted_message(poke_info):
    type_label = 'Types' if len(poke_info['types']) > 1 else 'Type'

    return """\
########################################################################################
# Index: %d
# Name: %s / %s (%s)
# Caregory: %s
# %s: %s
########################################################################################
    """ % (
        poke_info['core']['index'],
        poke_info['core']['name'],
        poke_info['core']['jp_name'],
        poke_info['core']['jp_rom_name'],
        poke_info['core']['category'],
        type_label,
        ' / '.join(poke_info['types'])
    )


---
&nbsp;
- ### Recriar arquivos dado o nome.

In [24]:
import os

def recreate_file(file_name):
    if (os.path.exists(file_name)):
        os.remove(file_name)

    open(file_name, 'x')



# Core Functions
---

- ### Carregar a página destino e retornar o soup.

In [25]:
from urllib.request import urlopen as uOpen, Request as uReq
from bs4 import BeautifulSoup as soup

def get_poke_soup(link):
    uClient         = uReq(link, headers={'User-Agent': 'Magic Browser'})
    uCon            = uOpen(uClient)
    poke_page_html  = uCon.read()
    uCon.close()
    
    return soup(poke_page_html, 'html.parser')

---
&nbsp;
- ### Checar se é a última página a ser verificada.
    - Em alguns casos, quando tá perto do lançamento de um novo jogo e só se tem informação de um ou mais pokémons, às vezes o Bulbapédia coloca informações não oficiais com um layout diferente.

In [26]:
def check_last_page(poke_soup):
    return poke_soup.find(id='mw-content-text').table.a['href'] == "/wiki/File:BulbaShadow.png"

---
&nbsp;
- ### Pegar informações do Pokémon
    - Essa é a função core, todas as funções de extração de dados são chamadas aqui.

In [27]:
def get_poke_info(poke_soup):
    info_table = poke_soup                      \
                    .find(id='mw-content-text') \
                    .find_all('table', recursive=False)[1]

    core  = get_core_poke_info(info_table)
    types = get_poke_types(info_table)
    
    return {
        'core': {
            'index':       core[0],
            'name':        core[1],
            'category':    core[2],
            'jp_name':     core[3],
            'jp_rom_name': core[4]
        },
        'types': types
    }

# Data Extraction Functions
---

- ### Pegar o link do próximo pokémon
    - Nessa parte da página, o Bulbapédia pode colocar trs a mais pra informar algo (geralmente relacionado ao Pokemon Go), por isso esse tratamento.

In [28]:
def get_next_pokemon_link(poke_soup):
    npl = poke_soup.find(id='mw-content-text').table
    try:
        npl = npl.findChildren('tr', recursive=False)[1]    \
                 .findChildren('td', recursive=False)[2]
    except IndexError:
        npl = npl.findChildren('tr', recursive=False)[0]    \
                 .findChildren('td', recursive=False)[2]
    finally:
        return npl.find('a')['href']            

---
&nbsp;
- ### Pegar informações core do pokémon:
    - Index (número na NationalDex)
    - Nome
    - Categoria
    - Nome (JP)
    - Nome (JP/Romaji)

In [29]:
def get_core_poke_info(info_table):
    base_info_container = info_table.tr.td.table.tr

    info_container = base_info_container.td.table.tr      \
                        .find_all('td', recursive=False)
    
    return (
        int(base_info_container.th.find('a').text.replace('#', '')), # index

        info_container[0].big.text,                                  # name
        info_container[0].a.text,                                    # category
        info_container[1].span.text,                                 # jp_name
        info_container[1].i.text                                     # jp_rom_name
    )

---
&nbsp;
- ### Pegar tipos do pokémon.

In [30]:
def get_poke_types(info_table):
    types = info_table.find_all('tr', recursive=False)[1]           \
                        .table.find('td', attrs={'style': None})    \
                        .find_all('a')
    
    return [t.text for t in types if t.text != 'Unknown']


# Main Cycle
---

- ### Constantes

In [31]:
    BASE_URL  = 'https://bulbapedia.bulbagarden.net'
    END_URL   = '/wiki/%3F%3F%3F_(Pok%C3%A9mon)'
    FILE_NAME = 'pokemon_list.txt'

---
&nbsp;
- ### Main

In [32]:
    recreate_file(FILE_NAME)

    with open(FILE_NAME, 'w') as f:
        next_pokemon_link = '/wiki/Bulbasaur_%28Pok%C3%A9mon%29'

        while(True):
            poke_soup = get_poke_soup(f'{BASE_URL}{next_pokemon_link}')
            if check_last_page(poke_soup):
                break
            
            poke_info = get_poke_info(poke_soup)
            message   = get_formatted_message(poke_info)

            f.write(message + '\n')
            print(message)

            next_pokemon_link = get_next_pokemon_link(poke_soup)
            if (next_pokemon_link == END_URL):
                break
            

        print("\n\n########## FINISHED ##########\n\n")


########################################################################################
# Index: 1
# Name: Bulbasaur / フシギダネ (Fushigidane)
# Caregory: Seed Pokémon
# Types: Grass / Poison
########################################################################################
    
########################################################################################
# Index: 2
# Name: Ivysaur / フシギソウ (Fushigisou)
# Caregory: Seed Pokémon
# Types: Grass / Poison
########################################################################################
    
########################################################################################
# Index: 3
# Name: Venusaur / フシギバナ (Fushigibana)
# Caregory: Seed Pokémon
# Types: Grass / Poison
########################################################################################
    
########################################################################################
# Index: 4
# Name: Charmander / ヒトカゲ (Hitokage)
# Caregory: Lizard

########################################################################################
# Index: 31
# Name: Nidoqueen / ニドクイン (Nidoqueen)
# Caregory: Drill Pokémon
# Types: Poison / Ground
########################################################################################
    
########################################################################################
# Index: 32
# Name: Nidoran♂ / ニドラン♂ (Nidoran♂)
# Caregory: Poison Pin Pokémon
# Type: Poison
########################################################################################
    
########################################################################################
# Index: 33
# Name: Nidorino / ニドリーノ (Nidorino)
# Caregory: Poison Pin Pokémon
# Type: Poison
########################################################################################
    
########################################################################################
# Index: 34
# Name: Nidoking / ニドキング (Nidoking)
# Caregory: Drill Pokémon

########################################################################################
# Index: 61
# Name: Poliwhirl / ニョロゾ (Nyorozo)
# Caregory: Tadpole Pokémon
# Type: Water
########################################################################################
    
########################################################################################
# Index: 62
# Name: Poliwrath / ニョロボン (Nyorobon)
# Caregory: Tadpole Pokémon
# Types: Water / Fighting
########################################################################################
    
########################################################################################
# Index: 63
# Name: Abra / ケーシィ (Casey)
# Caregory: Psi Pokémon
# Type: Psychic
########################################################################################
    
########################################################################################
# Index: 64
# Name: Kadabra / ユンゲラー (Yungerer)
# Caregory: Psi Pokémon
# Type: Psychic
###

########################################################################################
# Index: 91
# Name: Cloyster / パルシェン (Parshen)
# Caregory: Bivalve Pokémon
# Types: Water / Ice
########################################################################################
    
########################################################################################
# Index: 92
# Name: Gastly / ゴース (Ghos)
# Caregory: Gas Pokémon
# Types: Ghost / Poison
########################################################################################
    
########################################################################################
# Index: 93
# Name: Haunter / ゴースト (Ghost)
# Caregory: Gas Pokémon
# Types: Ghost / Poison
########################################################################################
    
########################################################################################
# Index: 94
# Name: Gengar / ゲンガー (Gangar)
# Caregory: Shadow Pokémon
# Types: Ghost / 

########################################################################################
# Index: 121
# Name: Starmie / スターミー (Starmie)
# Caregory: Mysterious Pokémon
# Types: Water / Psychic
########################################################################################
    
########################################################################################
# Index: 122
# Name: Mr. Mime / バリヤード (Barrierd)
# Caregory: Barrier Pokémon
# Types: Psychic / Fairy
########################################################################################
    
########################################################################################
# Index: 123
# Name: Scyther / ストライク (Strike)
# Caregory: Mantis Pokémon
# Types: Bug / Flying
########################################################################################
    
########################################################################################
# Index: 124
# Name: Jynx / ルージュラ (Rougela)
# Caregory: Human 

...

########################################################################################
# Index: 691
# Name: Dragalge / ドラミドロ (Dramidoro)
# Caregory: Mock Kelp Pokémon
# Types: Poison / Dragon
########################################################################################
    
########################################################################################
# Index: 692
# Name: Clauncher / ウデッポウ (Udeppou)
# Caregory: Water Gun Pokémon
# Type: Water
########################################################################################
    
########################################################################################
# Index: 693
# Name: Clawitzer / ブロスター (Bloster)
# Caregory: Howitzer Pokémon
# Type: Water
########################################################################################
    
########################################################################################
# Index: 694
# Name: Helioptile / エリキテル (Erikiteru)
# Caregory: Generato

########################################################################################
# Index: 721
# Name: Volcanion / ボルケニオン (Volcanion)
# Caregory: Steam Pokémon
# Types: Fire / Water
########################################################################################
    
########################################################################################
# Index: 722
# Name: Rowlet / モクロー (Mokuroh)
# Caregory: Grass Quill Pokémon
# Types: Grass / Flying
########################################################################################
    
########################################################################################
# Index: 723
# Name: Dartrix / フクスロー (Fukuthrow)
# Caregory: Blade Quill Pokémon
# Types: Grass / Flying
########################################################################################
    
########################################################################################
# Index: 724
# Name: Decidueye / ジュナイパー (Junaiper)
# Car

########################################################################################
# Index: 751
# Name: Dewpider / シズクモ (Shizukumo)
# Caregory: Water Bubble Pokémon
# Types: Water / Bug
########################################################################################
    
########################################################################################
# Index: 752
# Name: Araquanid / オニシズクモ (Onishizukumo)
# Caregory: Water Bubble Pokémon
# Types: Water / Bug
########################################################################################
    
########################################################################################
# Index: 753
# Name: Fomantis / カリキリ (Karikiri)
# Caregory: Sickle Grass Pokémon
# Type: Grass
########################################################################################
    
########################################################################################
# Index: 754
# Name: Lurantis / ラランテス (Lalantes)
# Care

########################################################################################
# Index: 780
# Name: Drampa / ジジーロン (Jijilong)
# Caregory: Placid Pokémon
# Types: Normal / Dragon
########################################################################################
    
########################################################################################
# Index: 781
# Name: Dhelmise / ダダリン (Dadarin)
# Caregory: Sea Creeper Pokémon
# Types: Ghost / Grass
########################################################################################
    
########################################################################################
# Index: 782
# Name: Jangmo-o / ジャラコ (Jyarako)
# Caregory: Scaly Pokémon
# Type: Dragon
########################################################################################
    
########################################################################################
# Index: 783
# Name: Hakamo-o / ジャランゴ (Jyarango)
# Caregory: Scaly Pokémo

########################################################################################
# Index: 809
# Name: Melmetal / メルメタル (Melmetal)
# Caregory: Hex Nut Pokémon
# Type: Steel
########################################################################################
    


########## FINISHED ##########


